In [11]:
import numpy as np
import pandas as pd
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

In [13]:
#scrape restaurant names
HTML_list = []
for i in np.arange(0,1000,10):
    url = 'https://www.yelp.com/search?find_loc=San+Francisco,+CA,+USA&start=%s&cflt=restaurants' %i
    #print url
    response = requests.get(url)
    HTML_list.append(response.text)

results_list = []
for page in HTML_list:
    results = Selector(text=page).xpath("//span[@class='indexed-biz-name']/a[@class='biz-name js-analytics-click']/span")
    results_list.append(results)
    
restaurant_name_list = []
for result in results_list:
        for name in result:
            restaurant_name_list.append(name.xpath("text()").extract())

In [14]:
#scrape bar names
HTML_list = []
for i in np.arange(0,1000,10):
    url = 'https://www.yelp.com/search?find_loc=San+Francisco,+CA,+USA&start=%s&cflt=bars' %i
    #print url
    response = requests.get(url)
    HTML_list.append(response.text)

bar_results_list = []
for page in HTML_list:
    bar_results = Selector(text=page).xpath("//span[@class='indexed-biz-name']/a[@class='biz-name js-analytics-click']/span")
    bar_results_list.append(bar_results)
    
bar_name_list = []
for result in bar_results_list:
        for bar in result:
            bar_name_list.append(bar.xpath("text()").extract())


In [19]:
restaurant_name_list = pd.Series(restaurant_name_list)
restaurant_name_list = restaurant_name_list.apply(lambda x: x[0].encode('ascii', 'ignore').rstrip().lstrip().replace(' ','-').replace('.','').replace("\u2019",''))

In [15]:
bar_name_list = pd.Series(bar_name_list)
bar_name_list = bar_name_list.apply(lambda x: x[0].encode('ascii', 'ignore').rstrip().lstrip().replace(' ','-').replace('.','').replace("\u2019",''))

In [16]:
bar_name_list

0                       Benjamin-Cooper
1            The-Black-Horse-London-Pub
2                        Smugglers-Cove
3                            Holy-Water
4                      The-Social-Study
5                                   ABV
6                           The-Alembic
7                            Third-Rail
8                          Fireside-Bar
9                             Blackbird
10                         High-Treason
11           Ferment-Drink-Repeat---FDR
12              Speakeasy-Ales-&-Lagers
13                            Shotwells
14                            Buzzworks
15               Alchemist-Bar-&-Lounge
16                              Bar-821
17                        The-Beer-Hall
18                              Bar-587
19                           Bloodhound
20                            Trick-Dog
21               Pacific-Cocktail-Haven
22                               Oddjob
23                         Horsefeather
24                        Valley-Tavern


In [20]:
restaurant_name_list

0                                 Ichido
1                           b-Patisserie
2                       Cuisine-of-Nepal
3                            Casa-Flores
4                                Tacorea
5                                Frances
6                                Dakshin
7                              Sugarfoot
8                  State-Bird-Provisions
9               Brendas-French-Soul-Food
10                                  Nopa
11                               Paprika
12                               Ro-Cafe
13              Italian-Homemade-Company
14                          Rove-Kitchen
15                           El-Farolito
16                          No-No-Burger
17                                   Lol
18                               Hogwash
19                            Chez-Maman
20                          No-No-Burger
21                                   Lol
22                             Nightbird
23                               Hogwash
24              

In [154]:
# Please only run this frame once to avoid hitting the site too hard all at once
def yelp_func(search_key):
    import requests

    # Search parameters -- for convenience 

    # Search and location get replaced in the next few lines

    HTML_list = []

    for i in ["?start=%d" %d for d in range(0,20,20)]:
        url = search_key+"%s" %i
        #print url
        response = requests.get(url)
        HTML_list.append(response.text)

    reviews_list = []
    from scrapy.selector import Selector
    from scrapy.http import HtmlResponse

    results_list = []
    for page in HTML_list:
    # This xpath query will find the number of total results
        results = Selector(text=page).xpath("//div[@class='review-wrapper']/div[@class='review-content']/p")
        results_list.append(results)

    for result in results_list:
        for review in result:
            reviews_list.append(review.xpath("text()").extract())
    #total_results = int(results[0].extract())
    #total_results


    southern_pacific = []
    for i in reviews_list:
        southern_pacific.append(i)
    return southern_pacific

In [153]:
scraped_reviews = {}
for i in restaurant_name_list:
    try:
        scraped_reviews[i] = yelp_func('https://www.yelp.com/biz/%s-san-francisco' %i)
    except: 
        continue
      

In [152]:
df = pd.DataFrame(np.array([scraped_reviews[i] for i in scraped_reviews.keys()]), scraped_reviews.keys())

,0
Sugarfoot,[[Sugarfoot consistently has delectable goodie...
Dakshin,[[5 stars well deserved!! This place is freaki...
Brendas-French-Soul-Food,[[SOUL food is right! I've raved about Brenda'...
Tacorea,"[[Found this place on Yelp, and had to check i..."
State-Bird-Provisions,[[This place is so amazing i don't even know w...
Frances,[[A solid 5-stars. A farm to table approach- e...
Cuisine-of-Nepal,"[[Ordered pick up on Yelp Eat24. , I got the r..."
Casa-Flores,[[I was looking around to see what to do for m...
Ichido,"[[Fairly insane. Never done a meal like this, ..."
b-Patisserie,[[these macaroons are the crack candy of macar...


In [118]:
pd.Series([scraped_reviews[i] for i in scraped_reviews.keys()])

0    [[Sugarfoot consistently has delectable goodie...
1    [[5 stars well deserved!! This place is freaki...
2    [[SOUL food is right! I've raved about Brenda'...
3    [[Found this place on Yelp, and had to check i...
4    [[This place is so amazing i don't even know w...
5    [[A solid 5-stars. A farm to table approach- e...
6    [[Ordered pick up on Yelp Eat24. , I got the r...
7    [[I was looking around to see what to do for m...
8    [[Fairly insane. Never done a meal like this, ...
9    [[these macaroons are the crack candy of macar...
dtype: object

In [75]:
import re
from nltk.corpus import stopwords # Import the stop word list

def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
    letters_only = raw_review.decode("utf-8", "strict")
    
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set

    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  

In [ ]:
short_cloud = {}
for i in new_cloud:
    try:
        new_cloud[i] = new_cloud[i].set_index('word').drop('xa').reset_index().sort_values('count', ascending = False)   
        short_cloud = new_cloud[i][0:20]
    except:
        new_cloud[i] = new_cloud[i].sort_values('count', ascending = False)
        short_cloud[i] = new_cloud[i][0:20]
